In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#read the txt file
data1 = pd.read_csv('/Users/Evelyn/Desktop/Hackthon/DataSet1__24Apr2018.txt',sep=' ')
data2 = pd.read_csv('/Users/Evelyn/Desktop/Hackthon/DataSet2_24Apr2018.txt',sep=' ')
data3 = pd.read_csv('/Users/Evelyn/Desktop/Hackthon/DataSet3__24Apr2018.txt',sep=' ')
data4 = pd.read_csv('/Users/Evelyn/Desktop/Hackthon/DataSet4__24Apr2018.txt',sep=' ')

In [3]:
#print the dimensions of four datasets
print(data1.shape)
print(data2.shape)
print(data3.shape)
print(data4.shape)

(178478, 24)
(711844, 3)
(5738, 6)
(23121, 6)


In [4]:
#deal with data1, which is the data provided by referral partner

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178478 entries, 0 to 178477
Data columns (total 24 columns):
CommunityCode       164549 non-null float64
CommunityZipCode    164549 non-null float64
ProspectID          178478 non-null object
AgeatInquiry        101838 non-null float64
CareLevel           169672 non-null object
Gender              174805 non-null object
Marital             17023 non-null object
Veteran             178478 non-null bool
CurrentResidence    36868 non-null object
ZipCode             63631 non-null float64
UnitPreference      6122 non-null object
ProvidedEmail       178478 non-null bool
ProvidedPhone       178478 non-null bool
Budget              111947 non-null object
RequestedVisit      178478 non-null bool
NoCall              178478 non-null bool
NoEmail             178478 non-null bool
InquiryDate         178421 non-null object
ReferralCompany     178478 non-null object
ProjectedMoveIn     102838 non-null object
DeactDate           108253 non-null object

In [5]:
data1.nunique()

CommunityCode          150
CommunityZipCode       136
ProspectID          178478
AgeatInquiry           109
CareLevel                5
Gender                   2
Marital                  4
Veteran                  2
CurrentResidence         4
ZipCode               7646
UnitPreference           6
ProvidedEmail            2
ProvidedPhone            2
Budget                   6
RequestedVisit           2
NoCall                   2
NoEmail                  2
InquiryDate         128427
ReferralCompany          5
ProjectedMoveIn      68716
DeactDate             7460
DeactReason              8
NotQual                  5
CurrentStatus            4
dtype: int64

In [6]:
data1.describe()

,CommunityCode,CommunityZipCode,AgeatInquiry,ZipCode
count,164549.000000,164549.000000,101838.000000,63631.000000
mean,594.087937,50576.659621,81.573214,51561.642957
std,180.166353,24091.052059,9.640544,24373.995451
min,272.000000,6413.000000,-31.000000,1002.000000
25%,458.000000,33021.000000,76.000000,33028.000000
50%,503.000000,53129.000000,83.000000,53129.000000
75%,740.000000,73135.000000,88.000000,76502.000000
max,899.000000,99324.000000,185.000000,99801.000000


In [7]:
data1.describe(include=['O'])

,ProspectID,CareLevel,Gender,Marital,CurrentResidence,UnitPreference,Budget,InquiryDate,ReferralCompany,ProjectedMoveIn,DeactDate,DeactReason,NotQual,CurrentStatus
count,178478,169672,174805,17023,36868,6122,111947,178421,178478,102838,108253,106579,13624,178478
unique,178478,5,2,4,4,6,6,128427,5,68716,7460,8,5,4
top,29f37977-7d24-e811-8117-e0071b669ee1,A,Female,Married,Living at Home,1 Bed,"3,000-4,000",2014-08-12 04:00:00,A Place for Mom,2016-12-01 05:00:00,2016-02-03 06:00:00,Request Removal,Not Financially Qualified,Deactivated
freq,1,72305,112344,8310,24875,3096,30917,103,105537,1036,333,32769,7219,108364


In [8]:
#Create new feature ConvLevel : group prospects into three Categories based on within what time frame to move in 
# 1: Hot, 2 : Warm, 3 : Cold
data1['ProjectedMoveIn'] = pd.to_datetime(data1.ProjectedMoveIn) 
data1['InquiryDate'] = pd.to_datetime(data1.InquiryDate)

data1['ConvLevel'] = data1.ProjectedMoveIn - data1.InquiryDate
data1['ConvLevel'] = data1['ConvLevel'].map(lambda x : x.days)

data1.loc[(data1.ConvLevel <= 30), 'ConvLevel'] = 1
data1.loc[(data1.ConvLevel > 30) & (data1.ConvLevel <= 90),'ConvLevel'] = 2
data1.loc[(data1.ConvLevel > 90),'ConvLevel'] = 3
data1['ConvLevel'] = data1.ConvLevel.fillna(3)
data1['ConvLevel'] = data1['ConvLevel'].astype(int)

In [9]:
# Add Conversion as the target/dependentable variable in data1
data1['Conversion'] = data1.ProspectID.isin(data3.ProspectID)
data1['Conversion'] = data1['Conversion'].map({True:1,False:0})

In [10]:
#Let's see the percentage of the each category in different features
data1[['CareLevel','Conversion']].groupby('CareLevel',as_index=False).mean()

,CareLevel,Conversion
0,A,0.033801
1,F,0.025260
2,I,0.029359
3,M,0.072201
4,S,0.011429


In [11]:
data1[['Budget','Conversion']].groupby('Budget',as_index=False).mean()

,Budget,Conversion
0,"0-1,000",0.031314
1,"1,000-2,000",0.027351
2,"2,000-3,000",0.027414
3,"3,000-4,000",0.033412
4,"4,000-5,000",0.034090
5,"5,000 +",0.034930


In [12]:
data1[['CurrentResidence','Conversion']].groupby('CurrentResidence',as_index=False).mean()

,CurrentResidence,Conversion
0,Living at Home,0.058171
1,Living with Family,0.071621
2,Rental Apartment,0.119725
3,Senior Community,0.100000


In [13]:
data1[['Gender','Conversion']].groupby('Gender',as_index=False).mean()

,Gender,Conversion
0,Female,0.032516
1,Male,0.032180


In [14]:
data1[['ReferralCompany','Conversion']].groupby('ReferralCompany',as_index=False).mean()

,ReferralCompany,Conversion
0,A Place for Mom,0.039190
1,Caring.com,0.018812
2,Other,0.108235
3,SeniorLiving.net,0.018695
4,Seniorhomes.com,0.024262


In [15]:
data1[['CurrentStatus','Conversion']].groupby('CurrentStatus',as_index=False).mean()

,CurrentStatus,Conversion
0,Deactivated,0.000434
1,Future,0.260797
2,Moved In,0.827686
3,Prospect,0.000429


In [16]:
data1[['ConvLevel','Conversion']].groupby('ConvLevel',as_index=False).mean()

,ConvLevel,Conversion
0,1,0.034176
1,2,0.129982
2,3,0.019092


In [17]:
data1[['Veteran','Conversion']].groupby('Veteran',as_index=False).mean()

,Veteran,Conversion
0,False,0.027816
1,True,0.037061


In [18]:
# Correcting and filling up the AgeatInquiry column, replacing the extreme age 185 with 85, 
#replacing age less than 21 and missing age with the mean age
data1.loc[data1.AgeatInquiry == 185, 'AgeatInquiry'] = 85
age_mean = data1['AgeatInquiry'].mean()
data1.loc[data1['AgeatInquiry']<18,'AgeatInquiry'] = age_mean
data1.AgeatInquiry = data1.AgeatInquiry.fillna(age_mean)

In [19]:
#Filling up the CareLevel with the most frequent one
CareLevel_freq = data1.CareLevel.dropna().mode()[0]
data1.CareLevel = data1.CareLevel.fillna(CareLevel_freq)

data1.CareLevel = data1.CareLevel.map({'A':1, 'I':2, 'F':3, 'S':4, 'M':5})

In [20]:
#Filling NAs with the most frequent one in Gender
#Converting Gender into categorical variable
data1.Gender = data1.Gender.fillna('Female')
data1.Gender = data1.Gender.map({'Female':1,'Male':2}).astype(int)

In [21]:
data1.Veteran = data1.Veteran.map({True:1, False:2})

In [22]:
data1.ProvidedEmail = data1.ProvidedEmail.map({True:1, False:2})
data1.ProvidedPhone = data1.ProvidedPhone.map({True:1, False:2})
data1.RequestedVisit = data1.RequestedVisit.map({True:1, False:2})
data1.NoCall = data1.NoCall.map({True:1, False:2})
data1.NoEmail = data1.NoEmail.map({True:1, False:2})

In [23]:
budget_map = {'0-1,000':1, '1,000-2,000':2, '2,000-3,000':3, '3,000-4,000':4, '4,000-5,000':5, '5,000 +':6}
data1.Budget = data1.Budget.fillna(data1.Budget.mode()[0])
data1.Budget = data1.Budget.map(budget_map)

In [24]:
#creating a new flag feature that is 1 when NotQual is null
data1['IsQual'] = 1
data1.loc[data1.NotQual.isnull(),'IsQual'] = 0

In [25]:
refer_map = {'A Place for Mom':1, 'Caring.com':2, 'SeniorLiving.net':3, 'Seniorhomes.com':4,'Other':5}
#data1.ReferralCompany = data1.ReferralCompany.fillna(data1.ReferralCompany.mode()[0])
data1.ReferralCompany = data1.ReferralCompany.map(refer_map)

In [26]:
deact_map = {'Request Removal':1, 'Not Qualified':2, 'Location':3, 'Moved to Other Community':4, 'Duplicate Lead':5,
            'Deceased':6, 'Health Status Changed':7, 'Moved In with Family':8}
data1.DeactReason = data1.DeactReason.fillna(data1.DeactReason.mode()[0])
data1.DeactReason = data1.DeactReason.map(deact_map)

In [27]:
status_map = {'Deactivated':1, 'Prospect':2, 'Moved In':3, 'Future':4}
data1.CurrentStatus = data1.CurrentStatus.map(status_map)

In [28]:
var = ['AgeatInquiry', 'CareLevel', 'Gender', 'Veteran', 'ProvidedEmail', 'ProvidedPhone', 'Budget', 'RequestedVisit',
      'NoCall', 'NoEmail', 'ReferralCompany', 'DeactReason','CurrentStatus']
x_all = data1[var]
y_all = data1['Conversion']

In [29]:
num_test = 0.2
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=num_test, random_state = 3843)

In [30]:
linear = LogisticRegression()
linear.fit(x_train,y_train)
pred = linear.predict(x_test)
print(linear.score(x_train,y_train))

0.9904960008964715
